In [2]:
import json
output = [
    "../data/similarity/gemma-1.1-2b-it.json", 
    "../data/similarity/gemma-1.1-7b-it.json", 
    "../data/similarity/gpt-3.5-turbo-instruct-0914.json", 
    "../data/similarity/Meta-Llama-3-8B-Instruct.json", 
    "../data/similarity/Phi-3-mini-4k-instruct.json", 
    "../data/similarity/Phi-3-mini-128k-instruct.json", 
    "../data/similarity/Mistral-7B-Instruct-v0.2.json",
    "../data/similarity/Mistral-7B-Instruct-v0.3.json"
]

output1 = [
    "../data/avg_similarity/gemma-1.1-2b-it.json", 
    "../data/avg_similarity/gemma-1.1-7b-it.json", 
    "../data/avg_similarity/gpt-3.5-turbo-instruct-0914.json", 
    "../data/avg_similarity/Meta-Llama-3-8B-Instruct.json", 
    "../data/avg_similarity/Phi-3-mini-4k-instruct.json", 
    "../data/avg_similarity/Phi-3-mini-128k-instruct.json", 
    "../data/avg_similarity/Mistral-7B-Instruct-v0.2.json",
    "../data/avg_similarity/Mistral-7B-Instruct-v0.3.json",
]


for _,t in enumerate(output1):
    with open(output[_], 'r') as file:
        data = json.load(file)
    averages = {}
    for key, value in data.items():
        avg = sum(value) / len(value)
        averages[key] = avg

    with open(output1[_], 'w') as file:
        json.dump(averages, file, indent=4)

In [1]:
import pandas as pd
import re
from tqdm import tqdm
import json

def get_edit_distance(sentence1, sentence2):
    ''' Sentence 1 is the original sentence with context addition poinsts and sentence 2 is the context-generated sentence by the LM '''
    m = len(sentence1)
    n = len(sentence2)
    
    ''' Find the Longest Common Subsequence of two strings'''
    
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if sentence1[i - 1] == sentence2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
    
    lcs = []
    i, j = m, n
    while i > 0 and j > 0:
        if sentence1[i - 1] == sentence2[j - 1]:
            lcs.append(sentence1[i - 1])
            i -= 1
            j -= 1
        elif dp[i - 1][j] > dp[i][j - 1]:
            i -= 1
        else:
            j -= 1
    lcs.reverse()
    
    
    ''' Find the minimum number of operations to convert sentence1 to sentence2 using the Longest Common Subsequence'''
    y=0
    count1 =0
    count2 =0
    for i in lcs:
        list1 = []
        list2 = []
        for j in range(count1,len(sentence1)):
            if i!=sentence1[j]:
                count1 +=1
                list1.append(sentence1[j])
            else:
                count1 +=1
                break
        for j in range(count2,len(sentence2)):
            if i!=sentence2[j]:
                count2 +=1
                list2.append(j)
            else:
                count2 +=1
                break
        if "___" in list1:
            y+=(len(list1) - 1)
        else:
            y+=len(list1) + len(list2)

    list1 = []
    list2 = []
    for j in range(count1,len(sentence1)):
        if i!=sentence1[j]:
            count1 +=1
            list1.append(sentence1[j])
        else:
            count1 +=1
            break
    for j in range(count2,len(sentence2)):
        if i!=sentence2[j]:
            count2 +=1
            list2.append(j)
        else:
            count2 +=1
            break
    if "___" in list1:
        y+=(len(list1) - 1)
    else:
        y+=len(list1) +len(list2)
    
    ''' Find the minimum number of operations to convert sentence2 to sentence1 using the Longest Common Subsequence using backward iteration'''
    
    y1=0 
    lcs.reverse()
    sentence1.reverse()
    sentence2.reverse()
    
    count1 =0
    count2 =0
    for i in lcs:
        list1 = []
        list2 = []
        for j in range(count1,len(sentence1)):
            if i!=sentence1[j]:
                count1 +=1
                list1.append(sentence1[j])
            else:
                count1 +=1
                break
        for j in range(count2,len(sentence2)):
            if i!=sentence2[j]:
                count2 +=1
                list2.append(j)
            else:
                count2 +=1
                break
        if "___" in list1:
            y1+=(len(list1) - 1)
        else:
            y1+=len(list1)+len(list2)

    list1 = []
    list2 = []
    for j in range(count1,len(sentence1)):
        if i!=sentence1[j]:
            count1 +=1
            list1.append(sentence1[j])
        else:
            count1 +=1
            break
    for j in range(count2,len(sentence2)):
        if i!=sentence2[j]:
            count2 +=1
            list2.append(j)
        else:
            count2 +=1
            break
    if "___" in list1:
        y1+=(len(list1) - 1)
    else:
        y1+=len(list1) +len(list2)
    
    
    return min(y,y1)




In [2]:
with open("../data/edit_distance/gpt-3.5-turbo-instruct-0914.json", 'r') as file:
    edit_distance_scores  =  json.load(file)

with open("../data/avg_edit_distance/gpt-3.5-turbo-instruct-0914.json", 'r') as file2:
    average_edit_distance_scores =  json.load(file2)
    

In [7]:
df = pd.read_csv("../../context-generation/data/data_generated/gpt_3.5/gpt-3.5-turbo-instruct-0914_2.0.csv")
# print(df)
scores = []
for j in df['index']:
    edit_distance_sum = 0
    count =0
    for a in range(1, 11):
        try:
            edit_distance = get_edit_distance( df[f"context_points"][int(j)].split(' ') , df[f"Column_{int(a)}"][int(j)].replace('\n' , '').split(' '))
            count = count+1
        except:
            edit_distance =0
        edit_distance_sum += edit_distance
    avg_edit_dist_per_datapoint = edit_distance_sum / count
    scores.append(avg_edit_dist_per_datapoint)
edit_distance_scores["2.0"] = scores
average_edit_distance_scores["2.0"] = sum(scores) / len(scores)

In [8]:
with open("../data/edit_distance/gpt-3.5-turbo-instruct-0914.json", 'w') as file:
    json.dump(edit_distance_scores , file )

with open("../data/avg_edit_distance/gpt-3.5-turbo-instruct-0914.json", 'w') as file2:
    json.dump(average_edit_distance_scores ,file2)

In [2]:
import pandas as pd
df = pd.read_csv("../../context-generation/data/data_generated/gpt_3.5/gpt-3.5-turbo-instruct-0914_2.0.csv")
sum =0 
count =0
for i in df['context_points']:
    list_word = i.replace('_' , '').split(' ')
    sum+=len(list_word)
    count +=1
    
print(sum/count)
    

    

12.323876036665212


In [5]:

from deap import base, creator, tools, algorithms
import random

# Problem definition
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))  # Minimize both objectives
creator.create("Individual", list, fitness=creator.FitnessMin)  # Define individual structure

ModuleNotFoundError: No module named 'deap'

In [ ]:
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -10, 10)  # Decision variable range
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=1)  # Individual creation
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # Population creation